# Reshaping Data From Wide to Long Form

This article highlights various ways to reshape data from wide to long form in python [datatable](https://datatable.readthedocs.io/en/latest/).

## **Selecting Columns**

### The Basics

In [1]:
from datatable import dt, f, Type
from typing import Pattern, NamedTuple, Union
from collections import Counter, defaultdict
from itertools import compress, chain
import re
import numpy as np
import pandas as pd



In [2]:
from datatable import dt, f
from typing import Pattern, NamedTuple, Union
from collections import Counter, defaultdict
from itertools import compress, chain
import re
import numpy as np

class measure(NamedTuple):
    """tuple for more complex reshape."""
    column_names:Union[str, list]
    sep:Union[str, Pattern] = None
    pattern:Union[str, Pattern] = None

def melt(data, id_vars=None, measure_vars=None, variable_name = 'variable', value_name = 'value'):
    "Turns Frame from wide to long form."
    if id_vars:
        if not isinstance(id_vars, (str, list, tuple)):
            raise TypeError('id_vars should be one of str, list, tuple.')
        if isinstance(id_vars, str):
            id_vars = [id_vars]
        checks = set(id_vars).difference(data.names)
        if checks:
            raise ValueError(f'Labels {checks} in id_vars do not exist in the column names.')
        if not set(data.names).difference(id_vars):
            return data
        checks = [key for key,value in Counter(id_vars).items() if value > 1]
        if checks:
            raise ValueError(f"Labels {checks} are duplicated in id_vars.")
        if not measure_vars:
            measure_vars = [name for name in data.names if name not in id_vars]
    if measure_vars:
        if not isinstance(measure_vars, (str, list, tuple)):
            raise TypeError('measure_vars should be one of str, list, tuple.')
        if isinstance(measure_vars, str):
            measure_vars = [measure_vars]
        checks = set(measure_vars).difference(data.names)
        if checks:
            raise ValueError(f'Labels {checks} in measure_vars do not exist in the column names.')
        checks = [key for key,value in Counter(measure_vars).items() if value > 1]
        if checks:
            raise ValueError(f"Labels {checks} are duplicated in measure_vars.")
        if (not id_vars) and (len(measure_vars) < data.ncols):
            id_vars = [name for name in data.names if name not in measure_vars]
    else:
        measure_vars = data.names

    def reshape_no_dot(measure_vars, output, data, id_vars=None):
        values = []
        for frame in data[:, measure_vars]:
            frame.names = [value_name]
            values.append(frame)
        values = dt.rbind(values, force=True)
        if id_vars:
            id_vars = dt.repeat(data[:, id_vars], len(measure_vars))
            return dt.cbind([id_vars, output, values], force = True)
        return dt.cbind([output, values], force = True)

    def reshape_dot(column_names, data, measure_vars, output, id_vars=None):
        "reshape if '.value' is present in the column names."
        boolean = [True if ent == '.value' else False for ent in column_names]
        dot_value = [[*compress(extract, boolean)] for extract in output]
        if len(dot_value[0]) > 1:
            dot_value = ["".join(extract) for extract in dot_value]
        else:
            dot_value = [*chain.from_iterable(dot_value)]
        checks = set(dot_value)
        if id_vars and checks.intersection(id_vars):
            raise ValueError(
                f"The new column names associated with .value -> {checks} "
                "are duplicated in id_vars."
            )
        boolean = [not true for true in boolean]
        others = [tuple(compress(extract, boolean)) for extract in output]
        headers_for_others = [extract for extract in column_names if extract != '.value']
        measure_vars = [frame for frame in data[:, measure_vars]]
        out = defaultdict(list)
        for key, value_column, frame in zip(others, dot_value, measure_vars):
            frame.names = [value_column]
            out[key].append(frame)
        headers_for_others = [dt.Frame([key], names = headers_for_others) for key, _ in out.items()]
        out = [dt.cbind(frame, force = True) for _, frame in out.items()]
        out = [dt.cbind(dt.repeat(left, right.nrows), right, force = True) for left, right in zip(headers_for_others, out)]
        out = dt.rbind(out, force = True)
        if id_vars:
            id_vars = dt.repeat(data[:, id_vars], out.nrows//data.nrows)
            return dt.cbind([id_vars, out], force = True)
        return out

    if not isinstance(variable_name, (str, tuple, dict, Pattern)):
        raise TypeError('variable_name should be one of string, tuple, dictionary, regular expression.')

    if isinstance(variable_name, str):
        if not isinstance(value_name, str):
            raise TypeError('value_name should be a string.')
        if value_name == variable_name:
            raise ValueError(
                f"{value_name} is duplicated as variable_name. "
                f"Kindly provide a unique argument for {value_name}.")
        if id_vars: 
            if variable_name in id_vars:
                raise ValueError(
                    f"{variable_name} already exists as a label "
                    "in id_vars. Kindly provide a unique argument.")
            if value_name in id_vars:
                raise ValueError(
                    f"{value_name} already exists as a label "
                    "in id_vars. Kindly provide a unique argument.")

        output = dt.Frame({variable_name:measure_vars})
        output = output[np.repeat(range(output.nrows), data.nrows),:]
        return reshape_no_dot(measure_vars=measure_vars, output = output, data = data, id_vars = id_vars)


    if isinstance(variable_name, Pattern):
        if not re.compile(variable_name).groups:
            raise ValueError("The regex should have at least one group.")
        output = [re.search(variable_name, word) for word in measure_vars]
        no_matches = [word for word, match in zip(measure_vars, output) if not match]
        if no_matches:
            raise ValueError(
                f"There was no match for labels {no_matches} "
                "for the provided regular expression.")
        output = [entry.groupdict() for entry in output]
        checks = output[0].keys()
        if id_vars and set(checks).intersection(id_vars):
            raise ValueError(
                f"Labels {checks} already exist in id_vars. "
                "Kindly provide unique names for the named groups " 
                "in the regular expression."
                )
        output = dt.Frame(output)
        output = output[np.repeat(range(output.nrows), data.nrows),:]        
        return reshape_no_dot(measure_vars=measure_vars, output = output, data = data, id_vars = id_vars)

    if isinstance(variable_name, dict) :
        checks = set(variable_name).intersection(id_vars)
        if id_vars and checks:
            raise ValueError(
                f"Labels {checks} already exist in id_vars. "
                "Kindly provide keys for the dictionary "
                "that do not exist in id_vars."
                )
        for key, regex in variable_name.items():
            if not isinstance(key, str):
                raise TypeError(f"{key} should be a string.")
            if not isinstance(regex, (str, Pattern)):
                raise TypeError(
                    f"The value for {key} should be a regular expression, "
                    "or can be compiled into one."
                    )
            if re.compile(regex).groups:
                raise ValueError("The regex should not have any groups.")
        output = []
        for key, regex in variable_name.items():
            out = [word for word in measure_vars if re.search(regex, word)]
            if not out:
                raise ValueError(
                    f"There was no match for {key} for regex => {regex}"
                )            
            
            measure_vars = [word for word in measure_vars if word not in out]
            if len(out) == 1:
                frame.names = [key]
                output.append(frame)
            else:
                values = []
                for frame in data[:, out]:
                    frame.names = [key]
                    values.append(frame)
                output.append(dt.rbind(values, force = True))
        output = dt.cbind(output, force=True)
        if id_vars:
            id_vars = dt.repeat(data[:, id_vars], output.nrows//data.nrows)
            return dt.cbind([id_vars, output])
        return output
          
    if isinstance(variable_name, tuple):
        variable_name = measure(*variable_name)
        column_names, sep, pattern = variable_name
        if not column_names:
            raise ValueError("Kindly provide argument for column_names, in the variable_name tuple.")
        if not isinstance(column_names, (str, list)):
            raise TypeError('column_names should be one of string, list.')
        if isinstance(column_names, str):
            column_names = [column_names]
        if id_vars:
            checks = set(column_names)
            checks.discard(".value")
            checks = checks.intersection(id_vars)
            if checks:
                raise ValueError(
                    f"Labels {checks} already exist in id_vars. "
                    "Kindly provide unique column_names "
                    "that do not exist in id_vars."
                    )
        if not any((sep, pattern)):
            raise ValueError("Kindly provide one of sep or pattern.")
        if sep and pattern:
            raise ValueError("only one of sep or pattern should be provided.")
        if sep:
            if not isinstance(sep, (str, Pattern)):
                raise TypeError(
                    "sep should be a regular expression, "
                    "or can be compiled into one.")
            output = [re.split(sep, word) for word in measure_vars]
            checks = max(map(len, output))
            if len(column_names) != checks:
                raise ValueError(
                    f"The maximum number of splits for sep -> {sep} is {checks} "
                    f"while the number of labels in {column_names} "
                    f"is {len(column_names)}"
                )
            if '.value' not in column_names:
                output = [*map(tuple, output)]
                output = dt.Frame(output, names=column_names)
                output = output[np.repeat(range(output.nrows), data.nrows),:]        
                return reshape_no_dot(measure_vars=measure_vars, output = output, data = data, id_vars = id_vars)

            return reshape_dot(column_names, data, measure_vars, output, id_vars=id_vars)

        if pattern:
            if not isinstance(pattern, (str, Pattern)):
                raise TypeError(
                    "pattern should be a regular expression, "
                    "or can be compiled into one.")
            checks = re.compile(pattern).groups
            if not checks:
                raise ValueError("The regex should have at least one group.")
            if checks != len(column_names):
                raise ValueError(
                    "The number of groups in the regex "
                    "should match the number of labels in column_names. "
                    f"The number of groups in the regex is {len(checks)}, "
                    f"while the length of column_names is {len(column_names)}")
            output = [re.findall(pattern, word) for word in measure_vars]
            no_matches = [word for word, match in zip(measure_vars, output) if not match]
            if no_matches:
                raise ValueError(
                    f"There was no match for labels {no_matches} "
                    "for the provided regular expression.")
            output = [*chain.from_iterable(output)]
            if '.value' not in column_names:
                output = [*map(tuple, output)]
                output = dt.Frame(output, names=column_names)
                output = output[np.repeat(range(output.nrows), data.nrows),:]        
                return reshape_no_dot(measure_vars=measure_vars, output = output, data = data, id_vars = id_vars)

            return reshape_dot(column_names, data, measure_vars, output, id_vars=id_vars)



In [3]:
DT = dt.Frame(
            [
                {
                    "ID": 1,
                    "DateRange1Start": "1/1/90",
                    "DateRange1End": "3/1/90",
                    "Value1": 4.4,
                    "DateRange2Start": "4/5/91",
                    "DateRange2End": "6/7/91",
                    "Value2": 6.2,
                    "DateRange3Start": "5/5/95",
                    "DateRange3End": "6/6/96",
                    "Value3": 3.3,
                }
            ])

DT

,ID,DateRange1Start,DateRange1End,Value1,DateRange2Start,DateRange2End,Value2,DateRange3Start,DateRange3End,Value3
,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1/1/90,3/1/90,4.4,4/5/91,6/7/91,6.2,5/5/95,6/6/96,3.3


In [4]:
melt(DT, id_vars='ID', variable_name= (['.value', 'num', '.value'], None, '(.+)(\d)(.*)'))

,ID,num,DateRangeStart,DateRangeEnd,Value
,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1,1/1/90,3/1/90,4.4
1,1,2,4/5/91,6/7/91,6.2
2,1,3,5/5/95,6/6/96,3.3


In [5]:
DT = dt.Frame(
        {
            "id": [1, 2, 3],
            "M_start_date_1": [201709, 201709, 201709],
            "M_end_date_1": [201905, 201905, 201905],
            "M_start_date_2": [202004, 202004, 202004],
            "M_end_date_2": [202005, 202005, 202005],
            "F_start_date_1": [201803, 201803, 201803],
            "F_end_date_1": [201904, 201904, 201904],
            "F_start_date_2": [201912, 201912, 201912],
            "F_end_date_2": [202007, 202007, 202007],
        }
    )

DT

,id,M_start_date_1,M_end_date_1,M_start_date_2,M_end_date_2,F_start_date_1,F_end_date_1,F_start_date_2,F_end_date_2
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1,201709,201905,202004,202005,201803,201904,201912,202007
1,2,201709,201905,202004,202005,201803,201904,201912,202007
2,3,201709,201905,202004,202005,201803,201904,201912,202007


In [6]:
melt(DT, id_vars='id', variable_name=(["cod", ".value", "date", "num"], None, "(M|F)_(start|end)_(date)_(.+)"))

,id,cod,date,num,start,end
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1,M,date,1,201709,201905
1,2,M,date,1,201709,201905
2,3,M,date,1,201709,201905
3,1,M,date,2,202004,202005
4,2,M,date,2,202004,202005
5,3,M,date,2,202004,202005
6,1,F,date,1,201803,201904
7,2,F,date,1,201803,201904
8,3,F,date,1,201803,201904


In [7]:
DT = dt.Frame(
    {
        "off_loc": ["A", "B", "C", "D", "E", "F"],
        "pt_loc": ["G", "H", "I", "J", "K", "L"],
        "pt_lat": [
            100.07548220000001,
            75.191326,
            122.65134479999999,
            124.13553329999999,
            124.13553329999999,
            124.01028909999998,
        ],
        "off_lat": [
            121.271083,
            75.93845266,
            135.043791,
            134.51128400000002,
            134.484374,
            137.962195,
        ],
        "pt_long": [
            4.472089953,
            -144.387785,
            -40.45611048,
            -46.07156181,
            -46.07156181,
            -46.01594293,
        ],
        "off_long": [
            -7.188632000000001,
            -143.2288569,
            21.242563,
            40.937416999999996,
            40.78472,
            22.905889000000002,
        ],
    }
)

DT

,off_loc,pt_loc,pt_lat,off_lat,pt_long,off_long
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,A,G,100.075,121.271,4.47209,−7.18863
1,B,H,75.1913,75.9385,−144.388,−143.229
2,C,I,122.651,135.044,−40.4561,21.2426
3,D,J,124.136,134.511,−46.0716,40.9374
4,E,K,124.136,134.484,−46.0716,40.7847
5,F,L,124.01,137.962,−46.0159,22.9059


In [8]:
melt(DT, variable_name=(['set', '.value'], None, r"(.+)_(.+)"))

,set,loc,lat,long
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,off,A,121.271,−7.18863
1,off,B,75.9385,−143.229
2,off,C,135.044,21.2426
3,off,D,134.511,40.9374
4,off,E,134.484,40.7847
5,off,F,137.962,22.9059
6,pt,G,100.075,4.47209
7,pt,H,75.1913,−144.388
8,pt,I,122.651,−40.4561


In [9]:
DT = dt.Frame({'Location': ['Madrid', 'Madrid', 'Rome', 'Rome'],
 'Account': ['ABC', 'XYX', 'ABC', 'XYX'],
 'Y2019:MTD:January:Expense': [4354, 769867, 434654, 632556456],
 'Y2019:MTD:January:Income': [56456, 32556456, 5214, 46724423],
 'Y2019:MTD:February:Expense': [235423, 6785423, 235423, 46588]})

DT

,Location,Account,Y2019:MTD:January:Expense,Y2019:MTD:January:Income,Y2019:MTD:February:Expense
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,Madrid,ABC,4354,56456,235423
1,Madrid,XYX,769867,32556456,6785423
2,Rome,ABC,434654,5214,235423
3,Rome,XYX,632556456,46724423,46588


In [10]:
melt(DT, id_vars=['Location', 'Account'], variable_name=(['year','month', '.value'], None, r"Y(.+):MTD:(.{3}).+(Income|Expense)"))

,Location,Account,year,month,Expense,Income
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,Madrid,ABC,2019,Jan,4354,56456
1,Madrid,XYX,2019,Jan,769867,32556456
2,Rome,ABC,2019,Jan,434654,5214
3,Rome,XYX,2019,Jan,632556456,46724423
4,Madrid,ABC,2019,Feb,235423,NA
5,Madrid,XYX,2019,Feb,6785423,NA
6,Rome,ABC,2019,Feb,235423,NA
7,Rome,XYX,2019,Feb,46588,NA


In [11]:
DT

,Location,Account,Y2019:MTD:January:Expense,Y2019:MTD:January:Income,Y2019:MTD:February:Expense
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,Madrid,ABC,4354,56456,235423
1,Madrid,XYX,769867,32556456,6785423
2,Rome,ABC,434654,5214,235423
3,Rome,XYX,632556456,46724423,46588


In [12]:
DT = dt.Frame(
            {
                "country": ["United States", "Russia", "China"],
                "vault_2012_f": [
                    48.132,
                    46.366,
                    44.266,
                ],
                "vault_2012_m": [46.632, 46.866, 48.316],
                "vault_2016_f": [
                    46.866,
                    45.733,
                    44.332,
                ],
                "vault_2016_m": [45.865, 46.033, 45.0],
                "floor_2012_f": [45.366, 41.599, 40.833],
                "floor_2012_m": [45.266, 45.308, 45.133],
                "floor_2016_f": [45.999, 42.032, 42.066],
                "floor_2016_m": [43.757, 44.766, 43.799],
            }
        )

DT

,country,vault_2012_f,vault_2012_m,vault_2016_f,vault_2016_m,floor_2012_f,floor_2012_m,floor_2016_f,floor_2016_m
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,United States,48.132,46.632,46.866,45.865,45.366,45.266,45.999,43.757
1,Russia,46.366,46.866,45.733,46.033,41.599,45.308,42.032,44.766
2,China,44.266,48.316,44.332,45,40.833,45.133,42.066,43.799


In [13]:
melt(DT, id_vars='country', variable_name=(['event', 'year', 'gender'], '_'))

,country,event,year,gender,value
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,United States,vault,2012,f,48.132
1,Russia,vault,2012,f,46.366
2,China,vault,2012,f,44.266
3,United States,vault,2012,m,46.632
4,Russia,vault,2012,m,46.866
5,China,vault,2012,m,48.316
6,United States,vault,2016,f,46.866
7,Russia,vault,2016,f,45.733
8,China,vault,2016,f,44.332


In [14]:
measure(['event','country'], None, '_')

measure(column_names=['event', 'country'], sep=None, pattern='_')

In [15]:
DT = dt.Frame({'id': [0, 1],
 'Name': ['ABC', 'XYZ'],
 'code': [1, 2],
 'code1': [4, np.nan],
 'code2': [8, 5],
 'type': ['S', 'R'],
 'type1': ['E', np.nan],
 'type2': ['T', 'U']})


DT

,id,Name,code,code1,code2,type,type1,type2
,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,0,ABC,1,4,8,S,E,T
1,1,XYZ,2,NA,5,R,NA,U


In [16]:
melt(DT, id_vars = ['id', 'Name'], variable_name={'code_all':'^code', 'type_all':'^type'})

,id,Name,code_all,type_all
,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,0,ABC,1,S
1,1,XYZ,2,R
2,0,ABC,4,E
3,1,XYZ,NA,NA
4,0,ABC,8,T
5,1,XYZ,5,U


In [17]:
DT = dt.Frame(
            [
                {
                    "ID": 1,
                    "DateRange1Start": "1/1/90",
                    "DateRange1End": "3/1/90",
                    "Value1": 4.4,
                    "DateRange2Start": "4/5/91",
                    "DateRange2End": "6/7/91",
                    "Value2": 6.2,
                    "DateRange3Start": "5/5/95",
                    "DateRange3End": "6/6/96",
                    "Value3": 3.3,
                }
            ])

DT

,ID,DateRange1Start,DateRange1End,Value1,DateRange2Start,DateRange2End,Value2,DateRange3Start,DateRange3End,Value3
,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1/1/90,3/1/90,4.4,4/5/91,6/7/91,6.2,5/5/95,6/6/96,3.3


In [18]:
melt(DT, id_vars='ID', variable_name={"DateRangeStart":"Start$", 'DateRangeEnd':'End$', 'Value':'^Value'})

,ID,DateRangeStart,DateRangeEnd,Value
,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1/1/90,3/1/90,4.4
1,1,4/5/91,6/7/91,6.2
2,1,5/5/95,6/6/96,3.3


In [19]:
DT = dt.Frame(
            {
                "country": ["United States", "Russia", "China"],
                "vault_2012_f": [
                    48.132,
                    46.366,
                    44.266,
                ],
                "vault_2012_m": [46.632, 46.866, 48.316],
                "vault_2016_f": [
                    46.866,
                    45.733,
                    44.332,
                ],
                "vault_2016_m": [45.865, 46.033, 45.0],
                "floor_2012_f": [45.366, 41.599, 40.833],
                "floor_2012_m": [45.266, 45.308, 45.133],
                "floor_2016_f": [45.999, 42.032, 42.066],
                "floor_2016_m": [43.757, 44.766, 43.799],
            }
        )

DT

,country,vault_2012_f,vault_2012_m,vault_2016_f,vault_2016_m,floor_2012_f,floor_2012_m,floor_2016_f,floor_2016_m
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,United States,48.132,46.632,46.866,45.865,45.366,45.266,45.999,43.757
1,Russia,46.366,46.866,45.733,46.033,41.599,45.308,42.032,44.766
2,China,44.266,48.316,44.332,45,40.833,45.133,42.066,43.799


In [20]:
pat = r"(?P<event>[a-z]+)_(?P<year>\d+)_(?P<gender>.+)"
out = melt(DT, id_vars='country', variable_name=re.compile(pat))
out
#dt.Frame(out)


,country,event,year,gender,value
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,United States,vault,2012,f,48.132
1,Russia,vault,2012,f,46.366
2,China,vault,2012,f,44.266
3,United States,vault,2012,m,46.632
4,Russia,vault,2012,m,46.866
5,China,vault,2012,m,48.316
6,United States,vault,2016,f,46.866
7,Russia,vault,2016,f,45.733
8,China,vault,2016,f,44.332


In [21]:
DT = dt.Frame({'subject': [1, 2],
                   'A_target_word_gd': [1, 11],
                   'A_target_word_fd': [2, 12],
                   'B_target_word_gd': [3, 13],
                   'B_target_word_fd': [4, 14],
                   'subject_type': ['mild', 'moderate']})


DT

,subject,A_target_word_gd,A_target_word_fd,B_target_word_gd,B_target_word_fd,subject_type
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1,1,2,3,4,mild
1,2,11,12,13,14,moderate


In [22]:
pat = "(?P<cond>[A-Z]).*(?P<value_type>gd|fd)"
melt(DT, id_vars=["subject", "subject_type"], variable_name=re.compile(pat))

,subject,subject_type,cond,value_type,value
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1,mild,A,gd,1
1,2,moderate,A,gd,11
2,1,mild,A,fd,2
3,2,moderate,A,fd,12
4,1,mild,B,gd,3
5,2,moderate,B,gd,13
6,1,mild,B,fd,4
7,2,moderate,B,fd,14


In [23]:
DT = dt.Frame({
    'famid': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'birth': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'ht1': [2.8, 2.9, 2.2, 2, 1.8, 1.9, 2.2, 2.3, 2.1],
    'ht2': [3.4, 3.8, 2.9, 3.2, 2.8, 2.4, 3.3, 3.4, 2.9]
})

DT

,famid,birth,ht1,ht2
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1,2.8,3.4
1,1,2,2.9,3.8
2,1,3,2.2,2.9
3,2,1,2,3.2
4,2,2,1.8,2.8
5,2,3,1.9,2.4
6,3,1,2.2,3.3
7,3,2,2.3,3.4
8,3,3,2.1,2.9


In [24]:
DT = dt.repeat(DT, 10_000)
DT

,famid,birth,ht1,ht2
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1,2.8,3.4
1,1,2,2.9,3.8
2,1,3,2.2,2.9
3,2,1,2,3.2
4,2,2,1.8,2.8
5,2,3,1.9,2.4
6,3,1,2.2,3.3
7,3,2,2.3,3.4
8,3,3,2.1,2.9


In [25]:
melt(DT, id_vars = ['famid', 'birth'], variable_name=(['.value', 'age'], None, r"(.+)(.)"))

,famid,birth,age,ht
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1,1,2.8
1,1,2,1,2.9
2,1,3,1,2.2
3,2,1,1,2
4,2,2,1,1.8
5,2,3,1,1.9
6,3,1,1,2.2
7,3,2,1,2.3
8,3,3,1,2.1


In [26]:
DT = dt.Frame([(1,1,2,3,4,5,6), (2,7,8,9,10,11,12)],
                  names=['id', 'ax','ay','az','bx','by','bz'])


DT

,id,ax,ay,az,bx,by,bz
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1,1,2,3,4,5,6
1,2,7,8,9,10,11,12


In [27]:
melt(DT, id_vars='id', variable_name=(['name', '.value'], None, '(.)(.)'))

,id,name,x,y,z
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1,a,1,2,3
1,2,a,7,8,9
2,1,b,4,5,6
3,2,b,10,11,12


In [28]:
DT = dt.Frame([{'id': 1, 'A1g_hi': 2, 
                    'A2g_hi': 3, 'A3g_hi': 4, 
                    'A4g_hi': 5}])

DT

,id,A1g_hi,A2g_hi,A3g_hi,A4g_hi
,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1,2,3,4,5


In [29]:
melt(DT, id_vars='id', variable_name=(['time', '.value'], None, "A(.)g_(.+)"))

,id,time,hi
,▪,▪▪▪▪,▪▪▪▪
0,1,1,2
1,1,2,3
2,1,3,4
3,1,4,5


In [30]:
DT = dt.Frame(
    [{'Sony | TV | Model | value': 'A222',
  'Sony | TV | Quantity | value': 5,
  'Sony | TV | Max-quant | value': 10,
  'Panasonic | TV | Model | value': 'T232',
  'Panasonic | TV | Quantity | value': 1,
  'Panasonic | TV | Max-quant | value': 10,
  'Sanyo | Radio | Model | value': 'S111',
  'Sanyo | Radio | Quantity | value': 4,
  'Sanyo | Radio | Max-quant | value': 9},
 {'Sony | TV | Model | value': 'A234',
  'Sony | TV | Quantity | value': 5,
  'Sony | TV | Max-quant | value': 9,
  'Panasonic | TV | Model | value': 'S3424',
  'Panasonic | TV | Quantity | value': 5,
  'Panasonic | TV | Max-quant | value': 12,
  'Sanyo | Radio | Model | value': 'S1s1',
  'Sanyo | Radio | Quantity | value': 2,
  'Sanyo | Radio | Max-quant | value': 9},
 {'Sony | TV | Model | value': 'A4345',
  'Sony | TV | Quantity | value': 4,
  'Sony | TV | Max-quant | value': 9,
  'Panasonic | TV | Model | value': 'X3421',
  'Panasonic | TV | Quantity | value': 1,
  'Panasonic | TV | Max-quant | value': 11,
  'Sanyo | Radio | Model | value': 'S1s2',
  'Sanyo | Radio | Quantity | value': 4,
  'Sanyo | Radio | Max-quant | value': 10}]
)

DT

,Sony | TV | Model | value,Sony | TV | Quantity | value,Sony | TV | Max-quant | value,Panasonic | TV | Model | value,Panasonic | TV | Quantity | value,Panasonic | TV | Max-quant | value,Sanyo | Radio | Model | value,Sanyo | Radio | Quantity | value,Sanyo | Radio | Max-quant | value
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,A222,5,10,T232,1,10,S111,4,9
1,A234,5,9,S3424,5,12,S1s1,2,9
2,A4345,4,9,X3421,1,11,S1s2,4,10


In [31]:
melt(DT, variable_name=(["Manufacturer", "Device", ".value"], None, r"(.+)\|(.+)\|(.+)\|.*"))

,Manufacturer,Device,Model,Quantity,Max-quant
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,Sony,TV,A222,5,10
1,Sony,TV,A234,5,9
2,Sony,TV,A4345,4,9
3,Panasonic,TV,T232,1,10
4,Panasonic,TV,S3424,5,12
5,Panasonic,TV,X3421,1,11
6,Sanyo,Radio,S111,4,9
7,Sanyo,Radio,S1s1,2,9
8,Sanyo,Radio,S1s2,4,10


In [32]:
DT = dt.Frame({'first': ['John', 'Mary'],
                   'last': ['Doe', 'Bo'],
                   'height': [5.5, 6.0],
                   'weight': [130, 150]})
DT

,first,last,height,weight
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,John,Doe,5.5,130
1,Mary,Bo,6,150


In [33]:
melt(DT)

,variable,value
,▪▪▪▪,▪▪▪▪
0,first,John
1,first,Mary
2,last,Doe
3,last,Bo
4,height,5.5
5,height,6.0
6,weight,130
7,weight,150


In [34]:
melt(DT).stypes

(stype.str32, stype.str32)

In [35]:
melt(DT, id_vars=['first', 'last'])

,first,last,variable,value
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,John,Doe,height,5.5
1,Mary,Bo,height,6
2,John,Doe,weight,130
3,Mary,Bo,weight,150


In [36]:
url = 'https://raw.githubusercontent.com/tidyverse/tidyr/main/data-raw/billboard.csv'
DT = dt.fread(url)
measure_vars = [name for name in DT.names if name.startswith('wk')]
melt(DT, measure_vars=measure_vars, variable_name='week')

,year,artist,track,time,date.entered,week,value
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk1,91
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk1,81
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk1,57
5,2000,98^0,Give Me Just One Nig...,3:24,2000-08-19,wk1,51
6,2000,A*Teens,Dancing Queen,3:44,2000-07-08,wk1,97
7,2000,Aaliyah,I Don't Wanna,4:15,2000-01-29,wk1,84
8,2000,Aaliyah,Try Again,4:03,2000-03-18,wk1,59


In [37]:
DT = dt.Frame([{'A': 'a', 'B': 1, 'C': 2},
 {'A': 'b', 'B': 3, 'C': 4},
 {'A': 'c', 'B': 5, 'C': 6}])

DT

,A,B,C
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,a,1,2
1,b,3,4
2,c,5,6


In [38]:
melt(DT, id_vars='A', measure_vars='B')

,A,variable,value
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,a,B,1
1,b,B,3
2,c,B,5


In [39]:
melt(DT, id_vars='A', measure_vars=['B', 'C'])

,A,variable,value
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


In [40]:
melt(DT, id_vars='A', measure_vars='B', variable_name='myVarname', value_name='myValname')

,A,myVarname,myValname
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,a,B,1
1,b,B,3
2,c,B,5


In [41]:
DT = dt.Frame({'Name': ['Bob', 'John', 'Foo', 'Bar', 'Alex', 'Tom'], 
                   'Math': ['A+', 'B', 'A', 'F', 'D', 'C'], 
                   'English': ['C', 'B', 'B', 'A+', 'F', 'A'],
                   'Age': [13, 16, 16, 15, 15, 13]})

DT

,Name,Math,English,Age
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,Bob,A+,C,13
1,John,B,B,16
2,Foo,A,B,16
3,Bar,F,A+,15
4,Alex,D,F,15
5,Tom,C,A,13


In [42]:
melt(DT, id_vars = ['Name', 'Age'], measure_vars=['Math', 'English'], variable_name='Subject', value_name='Grade')

,Name,Age,Subject,Grade
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,Bob,13,Math,A+
1,John,16,Math,B
2,Foo,16,Math,A
3,Bar,15,Math,F
4,Alex,15,Math,D
5,Tom,13,Math,C
6,Bob,13,English,C
7,John,16,English,B
8,Foo,16,English,B


In [43]:
DT = dt.Frame({
        'asset1':list('acacac'),
         'asset2':[4]*6,
         'A':[7,8,9,4,2,3],
         'D':[1,3,5,7,1,0],
         'E':[5,3,6,9,2,4]
})

DT

,asset1,asset2,A,D,E
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,a,4,7,1,5
1,c,4,8,3,3
2,a,4,9,5,6
3,c,4,4,7,9
4,a,4,2,1,2
5,c,4,3,0,4


In [44]:
melt(DT, id_vars=['asset1', 'asset2'], measure_vars = ['A', 'D'], variable_name='c_name', value_name='Value')

,asset1,asset2,c_name,Value
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,a,4,A,7
1,c,4,A,8
2,a,4,A,9
3,c,4,A,4
4,a,4,A,2
5,c,4,A,3
6,a,4,D,1
7,c,4,D,3
8,a,4,D,5


In [45]:
DT = dt.Frame({'New York': [25]})

DT

,New York
,▪▪▪▪
0,25


In [46]:
melt(DT)

,variable,value
,▪▪▪▪,▪▪▪▪
0,New York,25


In [47]:
DT = dt.Frame({'New york': [25], 'Paris': [27], 'London': [30]})

melt(DT)

,variable,value
,▪▪▪▪,▪▪▪▪
0,New york,25
1,Paris,27
2,London,30


In [48]:
temperatures = dt.Frame({
    'city': ['New York', 'London', 'Paris', 'Berlin', 'Amsterdam'],
    'day1': [23, 25, 27, 26, 24],
    'day2': [22, 21, 25, 26, 23],
    'day3': [26, 25, 24, 27, 23],
    'day4': [23, 21, 22, 26, 27],
    'day5': [27, 26, 27, 24, 28]
})

melt(temperatures, id_vars = 'city')

,city,variable,value
,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,New York,day1,23
1,London,day1,25
2,Paris,day1,27
3,Berlin,day1,26
4,Amsterdam,day1,24
5,New York,day2,22
6,London,day2,21
7,Paris,day2,25
8,Berlin,day2,26


In [49]:
melt(temperatures)

,variable,value
,▪▪▪▪,▪▪▪▪
0,city,New York
1,city,London
2,city,Paris
3,city,Berlin
4,city,Amsterdam
5,day1,23
6,day1,25
7,day1,27
8,day1,26


In [50]:
DT = dt.Frame(
            {
                "country": ["United States", "Russia", "China"],
                "vault_2012_f": [
                    48.132,
                    46.366,
                    44.266,
                ],
                "vault_2012_m": [46.632, 46.866, 48.316],
                "vault_2016_f": [
                    46.866,
                    45.733,
                    44.332,
                ],
                "vault_2016_m": [45.865, 46.033, 45.0],
                "floor_2012_f": [45.366, 41.599, 40.833],
                "floor_2012_m": [45.266, 45.308, 45.133],
                "floor_2016_f": [45.999, 42.032, 42.066],
                "floor_2016_m": [43.757, 44.766, 43.799],
            }
        )

DT

,country,vault_2012_f,vault_2012_m,vault_2016_f,vault_2016_m,floor_2012_f,floor_2012_m,floor_2016_f,floor_2016_m
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,United States,48.132,46.632,46.866,45.865,45.366,45.266,45.999,43.757
1,Russia,46.366,46.866,45.733,46.033,41.599,45.308,42.032,44.766
2,China,44.266,48.316,44.332,45,40.833,45.133,42.066,43.799
